## Точные методы решения СЛАУ (LU-разложение, метод Холецкого). Регуляризация.

In [1]:
from numpy import sqrt, array, zeros, ones, dot, random, eye
from numpy.linalg import solve, cond, norm
from prettytable import PrettyTable

In [2]:
#метод квадратного корня (Холецкого)
class SquareRootMethod:
    def __init__(self, A, b):
        self.n = A.shape[0]
        self.A = A
        self.b = b
        self.L = zeros((self.n, self.n))

    def __sum_one(self, i_first):
        s = 0
        for i in range(i_first):
            s += self.L[i_first][i]**2
        return s

    def __L_ii(self, i):
        self.L[i][i] = sqrt(self.A[i][i] - self.__sum_one(i))

    def __sum_two(self, i_first, j_first):
        s = 0
        for i in range(i_first):
            s += self.L[i_first][i]*self.L[j_first][i]
        return s

    def __L_ij(self, i_first, n):
        for i in range(i_first+1, self.n):
            self.L[i][i_first] = (self.A[i][i_first] - self.__sum_two(i_first, i))/self.L[i_first][i_first]

    def matrix_L(self):
        for i in range(self.n):
            self.__L_ii(i)
            self.__L_ij(i, self.n)
        return self.L
    
    def solve_x(self):
        self.matrix_L()
        y = solve(self.L, self.b)
        x = solve(self.L.T, y)
        return x

In [3]:
def hilbert_matrix(n: int):
    matrix = zeros((n, n))
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            matrix[i - 1][j - 1] = 1 / (i + j - 1)
    return matrix

Подберём параметр $\alpha$ для уравнения $(A + \alpha E)x_\alpha = b + \alpha x_0$.

In [11]:
for size in [3, 8, 12]:
    x = ones(size);
    A = hilbert_matrix(size);
    b = dot(A, x);
    print('Матрица Гильберта', size, 'x', size)
    print('Число обусловленности =', cond(A))
    results = PrettyTable()
    results.field_names = ["α", "cond(A+αE)", "|x-x*|"]
    min_diff = 100
    
    for alpha in range(-12, 0):
        var_A = A + 10**alpha * eye(size)
        diff_norm = norm(x - SquareRootMethod(var_A, b + 10**alpha * x).solve_x())
        results.add_row([10**alpha, cond(var_A), diff_norm])
        if diff_norm < min_diff:
            min_diff = diff_norm
            min_alpha = 10**alpha
    
    print(results)
    print("Наилучшее значение α =", min_alpha)
    
    rand_x = 10 * random.sample(A.shape[0])
    print("Случайный вектор x =", rand_x)
    b = dot(A, rand_x)
    solutions = PrettyTable()
    solutions.field_names = ["Уравнение", "|x-x*|"]
    solutions.add_row(["Ax = b", norm(rand_x - SquareRootMethod(A, b).solve_x())])
    solutions.add_row(["(A + αE)x = b", norm(rand_x - SquareRootMethod(A + 10**min_alpha * eye(size), b).solve_x())])
    solutions.add_row(["(A + αE)x = b + αx_0", norm(rand_x - SquareRootMethod(A + 10**min_alpha * eye(size), 
                                                                       b + rand_x * 10**min_alpha).solve_x())])
    
    print(solutions)
    print('')

Матрица Гильберта 3 x 3
Число обусловленности = 524.0567775860644
+--------+--------------------+------------------------+
|   α    |     cond(A+αE)     |         |x-x*|         |
+--------+--------------------+------------------------+
| 1e-12  | 524.0567773914216  | 2.186096514667649e-14  |
| 1e-11  | 524.0567756396885  | 6.811378861017111e-15  |
| 1e-10  | 524.0567581223308  | 7.012005498302777e-15  |
| 1e-09  |  524.05658294879   | 3.583957442378127e-14  |
| 1e-08  | 524.0548312199161  | 1.4320155638209064e-14 |
| 1e-07  | 524.0373145763426  | 2.087101174121004e-14  |
| 1e-06  | 523.8622126469712  | 1.4486177091987837e-14 |
| 1e-05  | 522.1176200879473  | 2.0197517065530995e-14 |
| 0.0001 | 505.2913341588696  | 5.691449808321057e-15  |
| 0.001  | 382.20473054975383 | 4.002966042486721e-16  |
|  0.01  | 111.79009054314739 | 2.5823128856601336e-15 |
|  0.1   | 14.688460348645588 | 8.005932084973442e-16  |
+--------+--------------------+------------------------+
Наилучшее значение α =